In [1]:
from google.colab import files
uploaded = files.upload()

Saving news_headlines_train.csv to news_headlines_train.csv


In [5]:
from google.colab import files
uploaded = files.upload()

Saving news_headlines_test_sample_submission.csv to news_headlines_test_sample_submission.csv


In [2]:
import pandas as pd
import numpy as np
import nltk
import re

In [6]:
train_df = pd.read_csv("news_headlines_train.csv")
test_df = pd.read_csv("news_headlines_test_sample_submission.csv")

# Data Preprocessing

In [14]:
from nltk import PorterStemmer,WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')

def Preprocess(df):
  for i in df['text']:
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    i = tokenizer.tokenize(i)

    tokenized_text_1 = df['text'].apply(lambda x: x.split())

    ps = PorterStemmer()
    WL = WordNetLemmatizer()

    stemmed_text = tokenized_text_1.apply(lambda x: [ps.stem(i) for i in x])
    lemmatized_text = tokenized_text_1.apply(lambda x: [WL.lemmatize(i) for i in x])

    stop = stopwords.words('english')
    stemmed_text = stemmed_text.apply(lambda x: [i for i in x if i not in stop])
    lemmatized_text = lemmatized_text.apply(lambda x: [i for i in x if i not in stop])

    for i in range(len(stemmed_text)):
      stemmed_text[i] = ' '.join(stemmed_text[i])

    df['stemmed_text'] = stemmed_text

    for i in range(len(lemmatized_text)):
        lemmatized_text[i] = ' '.join(lemmatized_text[i])

    df['lemmatized_text'] = lemmatized_text

    df = df[df["stemmed_text"] != '']
    df = df[df["lemmatized_text"] != '']


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [15]:
Preprocess(train_df)

In [16]:
Preprocess(test_df)

In [17]:
train_df.head()

,text,sentiment,stemmed_text,lemmatized_text
0,"In addition , a further 29 employees can be la...",-1,"In addit , 29 employe laid notic whole workfor...","In addition , 29 employee laid notice whole wo..."
1,The authorisation is in force until the end of...,0,authoris forc end next annual gener meet repea...,The authorisation force end next Annual Genera...
2,The value of the deal was not disclosed .,0,valu deal wa disclos .,The value deal wa disclosed .
3,You need to be ready when the window opens up ...,0,"need readi window open , rosberg said .","You need ready window open , Rosberg said ."
4,Major Order in India Comptel Corporation has r...,1,major order india comptel corpor ha receiv sig...,Major Order India Comptel Corporation ha recei...


In [19]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [21]:
y_train_NN = train_df["sentiment"]

x_train_stemmed_NN = train_df["stemmed_text"]
x_test_stemmed_NN = test_df["stemmed_text"]

x_train_lemmatized_NN = train_df["lemmatized_text"]
x_test_lemmatized_NN = test_df["lemmatized_text"]

In [22]:
x_train_stemmed_NN.shape, y_train_NN.shape ,x_test_stemmed_NN.shape

((3193,), (3193,), (807,))

In [24]:
tokenize = Tokenizer()
tokenize.fit_on_texts(x_train_stemmed_NN.values)

x_train_stemmed = tokenize.texts_to_sequences(x_train_stemmed_NN)
x_test_stemmed = tokenize.texts_to_sequences(x_test_stemmed_NN)

tokenize.fit_on_texts(x_train_lemmatized_NN.values)

x_train_lemmatized = tokenize.texts_to_sequences(x_train_lemmatized_NN)
x_test_lemmatized = tokenize.texts_to_sequences(x_test_lemmatized_NN)

# Model 4: Neural Network

In [25]:
EMBEDDING_DIM = 100
unknown = len(tokenize.word_index)+1
model = Sequential()
model.add(Embedding(unknown, EMBEDDING_DIM))
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2 ))
model.add(Dense(5, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         944600    
_________________________________________________________________
lstm (LSTM)                  (None, 128)               117248    
_________________________________________________________________
dense (Dense)                (None, 5)                 645       
Total params: 1,062,493
Trainable params: 1,062,493
Non-trainable params: 0
_________________________________________________________________


In [29]:
x_train_stemmed = pad_sequences(x_train_stemmed).astype(float)
x_test_stemmed = pad_sequences(x_test_stemmed).astype(float)

x_train_lemmatized = pad_sequences(x_train_lemmatized)
x_test_lemmatized = pad_sequences(x_test_lemmatized)

In [31]:
def relabel(x):
    if (x == -1):
        return 2
    elif (x == 1):
        return 1
    elif (x == 0):
        return 0

y_train_NN = y_train_NN.apply(relabel)

Training with stemmed text

In [32]:
model.fit(x_train_stemmed, y_train_NN, batch_size=128, epochs=7, verbose=1)

Epoch 1/7
25/25 [==============================] - 6s 243ms/step - loss: 1.1443 - accuracy: 0.5775
Epoch 2/7
25/25 [==============================] - 6s 246ms/step - loss: 0.8775 - accuracy: 0.6044
Epoch 3/7
25/25 [==============================] - 6s 242ms/step - loss: 0.7279 - accuracy: 0.6887
Epoch 4/7
25/25 [==============================] - 6s 246ms/step - loss: 0.5437 - accuracy: 0.7670
Epoch 5/7
25/25 [==============================] - 6s 244ms/step - loss: 0.4100 - accuracy: 0.8265
Epoch 6/7
25/25 [==============================] - 6s 242ms/step - loss: 0.2867 - accuracy: 0.8713
Epoch 7/7
25/25 [==============================] - 6s 243ms/step - loss: 0.1746 - accuracy: 0.9396


In [33]:
pred = model.predict_classes(x_test_stemmed)
pred

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 2, 1, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 1, 2, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 1, 0, 1,
       0, 1, 1, 2, 0, 0, 1, 0, 0, 2, 0, 2, 1, 1, 0, 2, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 0, 2, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 1, 0, 2, 0, 2, 0, 1, 0, 1, 1, 1, 0, 2, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 2, 0, 0, 1, 0, 2, 1, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 2, 1, 1, 0, 1, 0, 0, 2, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0,
       2, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 0, 0, 0,
       1, 0, 2, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 2, 1, 0, 2, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 2, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1,

Training with lemmatized text

In [44]:
model.fit(x_train_lemmatized, y_train_NN, batch_size=128, epochs=7, verbose=1)

Epoch 1/7
25/25 [==============================] - 6s 247ms/step - loss: 1.1129 - accuracy: 0.5628
Epoch 2/7
25/25 [==============================] - 6s 244ms/step - loss: 0.7768 - accuracy: 0.6558
Epoch 3/7
25/25 [==============================] - 6s 246ms/step - loss: 0.6385 - accuracy: 0.7200
Epoch 4/7
25/25 [==============================] - 6s 246ms/step - loss: 0.4372 - accuracy: 0.8312
Epoch 5/7
25/25 [==============================] - 6s 246ms/step - loss: 0.2713 - accuracy: 0.9048
Epoch 6/7
25/25 [==============================] - 6s 248ms/step - loss: 0.1455 - accuracy: 0.9555
Epoch 7/7
25/25 [==============================] - 6s 248ms/step - loss: 0.0830 - accuracy: 0.9768


In [45]:
pred2 = model.predict_classes(x_test_lemmatized)
pred2

array([0, 0, 1, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 0, 0, 2, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 2, 0, 1, 0, 1, 0, 0, 1, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 1,
       0, 0, 1, 2, 0, 0, 1, 0, 0, 0, 0, 2, 0, 1, 0, 2, 0, 1, 0, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 0, 1, 0, 2, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 2, 0, 2, 1, 0, 0, 2, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 1, 0, 1, 1, 0, 0, 2,
       0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 2, 1, 0, 0, 0, 1,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 2, 1,
       0, 0, 0, 0, 0, 1, 2, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 2, 0, 1,
       2, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 1, 0, 0, 1, 2, 1, 0, 0, 0,
       1, 0, 2, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0,
       1, 0, 2, 0, 0, 0, 1, 0, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0,
       0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 1, 1, 0, 0, 2,

In [48]:
submit_df = pd.read_csv("news_headlines_test_sample_submission.csv", index_col=0)
submit_df2 = pd.read_csv("news_headlines_test_sample_submission.csv", index_col=0)

In [49]:
submit_df["sentiment"] = pred
submit_df.head()

,sentiment
text,
"Proline Plus is available in both adjustable single and multichannel models and fixed volume single-channel models , in volume ranges from 0.1 micro litres to 10 ml .",0
"Digia said its consolidated net sales for January-June 2010 were EUR67 .8 m , up 9.7 % on the same period in 2009 ( EUR61 .9 m ) .",0
Cash flow from operating activities is estimated to be positive .,1
11 August 2010 - Finnish measuring equipment maker Vaisala Oyj HEL : VAIAS said today that its net loss widened to EUR4 .8 m in the first half of 2010 from EUR2 .3 m in the corresponding period a year earlier .,2
"Metso Foundries Jyvaskyla Oy will discontinue production on this line by 30 September 2008 , the company said .",0


Relabel negative sentiments back to -1

In [50]:
def relabel_back(x):
    if (x == 2):
        return -1
    elif (x == 1):
        return 1
    elif (x == 0):
        return 0

submit_df['sentiment'] = submit_df['sentiment'].apply(relabel_back)

In [52]:
submit_df.to_csv("NNstemmed.csv")

In [47]:
submit_df2["sentiment"] = pred2
submit_df2.head()

,text,sentiment
0,Proline Plus is available in both adjustable s...,0
1,Digia said its consolidated net sales for Janu...,0
2,Cash flow from operating activities is estimat...,1
3,11 August 2010 - Finnish measuring equipment m...,2
4,Metso Foundries Jyvaskyla Oy will discontinue ...,0


In [53]:
submit_df.to_csv("NNlemmatized.csv")

# Model 5: Decision Trees

Since lemmatized text seems to do better than stemmed text in the previous model, let's use lemmatized text here. 

In [71]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [98]:
count_vector = CountVectorizer()
x_train_counts = count_vector.fit_transform(train_df["text"].astype('U').values)

tfidf_transformer = TfidfTransformer()
x_train_tfidf = tfidf_transformer.fit_transform(x_train_counts.astype('U'))

x_test_counts = count_vector.transform(test_df["text"])
x_test_tfidf = tfidf_transformer.transform(x_test_counts)

In [102]:
from sklearn.tree import DecisionTreeClassifier

trees = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
DT = trees.fit(x_train_tfidf, train_df["sentiment"])
DTprediction = DT.predict(x_test_tfidf)

In [105]:
DTprediction

array([ 0,  0,  1,  1,  1,  1,  0,  0,  0,  1, -1,  0,  1,  0,  1,  0,  0,
        0,  0,  1,  0,  0, -1, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        1,  1,  0,  1,  1,  0,  0,  1,  0, -1, -1,  0, -1, -1,  0,  0,  0,
        0,  1, -1,  0, -1,  0,  0,  0,  0,  0,  1,  1,  1,  0,  0,  0,  0,
       -1, -1,  1,  0,  0,  0,  0,  0,  1, -1,  0,  0,  0,  1,  0,  0,  0,
        0,  0,  1,  0,  1,  1,  0,  0,  0,  1,  1,  0,  0,  0,  0,  0,  1,
        0,  0,  0,  0,  0, -1,  1,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,
        0,  1, -1,  1,  0, -1,  0, -1, -1,  0,  0,  0,  0,  0,  0,  1,  0,
        0,  0,  0, -1,  0,  1,  0, -1,  1,  0,  0,  0,  0,  1,  1,  0,  0,
        1,  0,  0,  0, -1,  0,  1,  0,  1,  0,  1,  0,  0,  0,  0,  0,  0,
       -1,  0,  0,  0,  0,  0, -1,  1,  1,  1,  0,  1,  0,  1,  1,  0,  0,
        0,  0, -1,  0,  0, -1,  0,  1,  0,  1,  0,  0,  1,  1,  0,  1,  1,
        1,  1,  0,  1,  1,  0,  0,  0,  1,  0,  0,  1,  0, -1,  0,  1,  1,
        0,  0,  0,  1,  0

In [106]:
submit_df3 = pd.read_csv("news_headlines_test_sample_submission.csv", index_col=0)


In [107]:
submit_df3["sentiment"] = DTprediction
submit_df3.head()

,sentiment
text,
"Proline Plus is available in both adjustable single and multichannel models and fixed volume single-channel models , in volume ranges from 0.1 micro litres to 10 ml .",0
"Digia said its consolidated net sales for January-June 2010 were EUR67 .8 m , up 9.7 % on the same period in 2009 ( EUR61 .9 m ) .",0
Cash flow from operating activities is estimated to be positive .,1
11 August 2010 - Finnish measuring equipment maker Vaisala Oyj HEL : VAIAS said today that its net loss widened to EUR4 .8 m in the first half of 2010 from EUR2 .3 m in the corresponding period a year earlier .,1
"Metso Foundries Jyvaskyla Oy will discontinue production on this line by 30 September 2008 , the company said .",1


In [108]:
submit_df3.to_csv("decisiontrees.csv")